In [1]:
import pandas as pd
import numpy as np


In [48]:
data=pd.read_csv('data.csv')
df=data.copy(deep=True)

In [49]:
df.head()

,id,sentence
0,321712,Hey 👋 \n\nWe re using our bot:\n\nhttps://t.me...
1,321713,Good stuff \n\nI am surprised I took so long t...
2,321717,you are using a non-official one
3,321718,use the one that uniswap uses: https://thegrap...
4,321719,keep in mind this is a hot subgraph so it can ...


In [50]:

df.rename(columns={'sentence':'text'},inplace=True)

In [51]:
df.head()

,id,text
0,321712,Hey 👋 \n\nWe re using our bot:\n\nhttps://t.me...
1,321713,Good stuff \n\nI am surprised I took so long t...
2,321717,you are using a non-official one
3,321718,use the one that uniswap uses: https://thegrap...
4,321719,keep in mind this is a hot subgraph so it can ...


In [54]:
#change object to string
df['text']=df['text'].astype(str)

In [58]:
def clean_text(text):
    text=text.lower()
    text=re.sub(r'[^a-z0-9\s]','',text)
    text=re.sub(r'\s+',' ',text)
    text=re.sub(r'^\s','',text)
    text=re.sub(r'\s$','',text)
    text=re.sub(r'http\S+','',text)
    return text

df['text']=df['text'].apply(clean_text)


In [59]:
df.head()

,id,text
0,321712,hey we re using our bot and also filtering su...
1,321713,good stuff i am surprised i took so long to fi...
2,321717,you are using a nonofficial one
3,321718,use the one that uniswap uses
4,321719,keep in mind this is a hot subgraph so it can ...


In [60]:
def remove_stopwords(text):
    text=text.split()
    text=[word for word in text if word not in stopwords.words('english')]
    text=' '.join(text)
    return text

df['text']=df['text'].apply(remove_stopwords)


In [6]:
df2=pd.read_csv('crypto_results.csv')
df2.head()

,sentence,tag
0,welcome to coinmarketcap.com! this site was fo...,1
1,"here at coinmarketcap, we work very hard to en...",1
2,each of our coin data pages has a graph that s...,1
3,we receive updated cryptocurrency prices direc...,1
4,related links new to crypto? learn how to bu...,1


In [9]:
df2.rename(columns={'sentence':'text2'},inplace=True)
df2.head()

,text2,tag
0,welcome to coinmarketcap.com! this site was fo...,1
1,"here at coinmarketcap, we work very hard to en...",1
2,each of our coin data pages has a graph that s...,1
3,we receive updated cryptocurrency prices direc...,1
4,related links new to crypto? learn how to bu...,1


In [10]:
import re
def clean_text(text2):
    text2=text2.lower()
    text2=re.sub(r'[^a-z0-9\s]','',text2)
    text2=re.sub(r'\s+',' ',text2)
    text2=re.sub(r'^\s','',text2)
    text2=re.sub(r'http\S+','',text2)
    return text2

df2['text2']=df2['text2'].apply(clean_text)
df2.head()

,text2,tag
0,welcome to coinmarketcapcom this site was foun...,1
1,here at coinmarketcap we work very hard to ens...,1
2,each of our coin data pages has a graph that s...,1
3,we receive updated cryptocurrency prices direc...,1
4,related links new to crypto learn how to buy b...,1


In [11]:
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer

def remove_stopwords(text):
    stop_words=set(stopwords.words('english'))
    tokens=nltk.word_tokenize(text)
    tokens=[token for token in tokens if token not in stop_words]
    return ' '.join(tokens)

df2['text2']=df2['text2'].apply(remove_stopwords)

In [12]:
def lemmatize_text(text):
    lemmatizer=WordNetLemmatizer()
    tokens=nltk.word_tokenize(text)
    tokens=[lemmatizer.lemmatize(token) for token in tokens]
    return ' '.join(tokens)

df2['text2']=df2['text2'].apply(lemmatize_text)

In [13]:
df2.head()

,text2,tag
0,welcome coinmarketcapcom site founded may 2013...,1
1,coinmarketcap work hard ensure relevant uptoda...,1
2,coin data page graph show current historic pri...,1
3,receive updated cryptocurrency price directly ...,1
4,related link new crypto learn buy bitcoin toda...,1


In [14]:
#train test split
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(df2['text2'], df2['tag'], test_size=0.2, random_state=42)



In [15]:
X_train=df2['text2']
y_train=df2['tag']

from sklearn.feature_extraction.text import TfidfVectorizer
tfidf=TfidfVectorizer()
X_train=tfidf.fit_transform(X_train)

from sklearn.naive_bayes import MultinomialNB
model=MultinomialNB()
model.fit(X_train,y_train)

X_test=tfidf.transform(X_test)
y_pred=model.predict(X_test)

from sklearn.metrics import accuracy_score
accuracy_score(y_test,y_pred)


0.8269689737470167

In [16]:
#svm
from sklearn.svm import SVC
model=SVC()
model.fit(X_train,y_train)
y_pred=model.predict(X_test)
accuracy_score(y_test,y_pred)

0.9928400954653938

In [17]:
#random forest
from sklearn.ensemble import RandomForestClassifier
model=RandomForestClassifier()
model.fit(X_train,y_train)
y_pred=model.predict(X_test)
accuracy_score(y_test,y_pred)

1.0

In [18]:
#neural network classifier 
from sklearn.neural_network import MLPClassifier
model=MLPClassifier()
model.fit(X_train,y_train)
y_pred=model.predict(X_test)






In [19]:
#check accuracy
from sklearn.metrics import accuracy_score
accuracy_score(y_test,y_pred)


1.0

In [20]:
#baggging
from sklearn.ensemble import BaggingClassifier
model=BaggingClassifier()
model.fit(X_train,y_train)
y_pred=model.predict(X_test)
accuracy_score(y_test,y_pred)


0.9928400954653938

In [21]:
#fine tuning
from sklearn.model_selection import GridSearchCV
model= BaggingClassifier()
params={'n_estimators':[10,20,30,40,50,60,70,80,90,100]}
grid=GridSearchCV(model,param_grid=params,cv=5)
grid.fit(X_train,y_train)
grid.best_params_
grid.best_score_




0.8521241620402449

In [24]:
print(grid.best_params_)
print(grid.best_score_)

#save model
import pickle
pickle.dump(model,open('model.pkl','wb'))


{'n_estimators': 70}
0.8521241620402449


In [36]:
#load model
model=pickle.load(open('model.pkl','rb'))


In [69]:
#test model usinmg new data.csv and 

X_test=df['text']
X_test=tfidf.transform(X_test)
y_pred=model.predict(X_test)
y_pred

df['tag']=y_pred
df.head()

df.to_csv('crypto_resultss.csv')




In [2]:
#pipeline for model
# from sklearn.pipeline import Pipeline
# pipeline=Pipeline([('tfidf',TfidfVectorizer()),('model',BaggingClassifier())])
# pipeline.fit(X_train,y_train)
# y_pred=pipeline.predict(X_test)
# accuracy_score(y_test,y_pred)
